<a href="https://colab.research.google.com/github/Rahulraj31/Intel-Image-Classification/blob/main/Intel_img_clas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import PIL.Image as Image
import cv2
import numpy as np
import pandas as pd

import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras
from tensorflow.keras import layers,losses
from tensorflow.keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator

In [1]:
import zipfile

local_zip = '/content/drive/MyDrive/Colab Notebooks/Intel_Img_Class.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [3]:
from pathlib import Path 
# Define path to the data directory
#dir_alldata = 

# Path to train directory 
test_data_dir = Path('/content/seg_pred/seg_pred')

# Path to validation directory
val_data_dir = Path("/content/seg_test/seg_test")

# Path to test directory
train_data_dir = Path("/content/seg_train/seg_train")

# Get the path to the normal and pneumonia sub-directories
"""normal_cases_train = train_data_dir / 'NORMAL'
pneumonia_cases_train = train_data_dir / 'PNEUMONIA'"""

"normal_cases_train = train_data_dir / 'NORMAL'\npneumonia_cases_train = train_data_dir / 'PNEUMONIA'"

In [4]:
train_images_dict = {
    'building': list(train_data_dir.glob('buildings/*')) ,   
    'forest': list(train_data_dir.glob('forest/*')),
    'glacier': list(train_data_dir.glob('glacier/*')),
    'mountain': list(train_data_dir .glob('mountain/*')),
    'sea': list(train_data_dir.glob('sea/*')),
    'street': list(train_data_dir.glob('street/*')), 
}


val_images_dict = {
     'building': list(val_data_dir.glob('buildings/*')) ,   
    'forest': list(val_data_dir.glob('forest/*')),
    'glacier': list(val_data_dir.glob('glacier/*')),
    'mountain': list(val_data_dir .glob('mountain/*')),
    'sea': list(val_data_dir.glob('sea/*')),
    'street': list(val_data_dir.glob('street/*')), 
}


labels_dict = {
     'building': "building", 
    'forest': "forest",
    'glacier': "glacier",
    'mountain': "mountain",
    'sea': "sea",
    'street': "street", 
}

In [8]:
img= cv2.imread(str(train_images_dict['building'][0]))
img.shape

(150, 150, 3)

In [9]:
Xtrain, ytrain = [], []
for name,images in train_images_dict.items():#returns key , path for each record
    for image in images:
        img=str(image)
    
        Xtrain.append(img)
        ytrain.append(labels_dict[name])


Fseries = pd.Series(Xtrain, name="filepaths",dtype=str)
Lseries = pd.Series(ytrain, name="labels",dtype=str)
train_data = pd.concat([Fseries,Lseries], axis=1)
train_df = pd.DataFrame(train_data)

print(train_df["labels"].value_counts())

mountain    2512
glacier     2404
street      2382
sea         2274
forest      2271
building    2191
Name: labels, dtype: int64


In [10]:
Xval, yval = [], []
for name,images in val_images_dict.items():#returns key , path for each record
    for image in images:
        img=str(image)
    
        Xval.append(img)
        yval.append(labels_dict[name])


Fseries = pd.Series(Xval, name="filepaths",dtype=str)
Lseries = pd.Series(yval, name="labels",dtype=str)
val_data = pd.concat([Fseries,Lseries], axis=1)
val_df = pd.DataFrame(val_data)

val_df["labels"].value_counts()

glacier     553
mountain    525
sea         510
street      501
forest      474
building    437
Name: labels, dtype: int64

In [12]:
#Generate batches of tensor image data with real-time data augmentation.

image_gen = ImageDataGenerator(rescale = 1./255,
                                   
                                   
                                   rotation_range = 40,
                                  shear_range = 0.2,
                                   zoom_range = 0.2,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   horizontal_flip = True,
                                   fill_mode = 'nearest'
    
)

train = image_gen.flow_from_dataframe(dataframe= train_df,x_col="filepaths",y_col="labels",
                                      target_size=(150,150),
                                     
                                      class_mode="categorical", #used for Sequential Model
                                      batch_size=32,
                                      shuffle=False            #do not shuffle data
                                     )

val = image_gen.flow_from_dataframe(dataframe= val_df,x_col="filepaths", y_col="labels",
                                    target_size=(150,150),
                                   
                                    class_mode="categorical",
                                    batch_size=32,
                                    shuffle=False
                                   )

Found 14034 validated image filenames belonging to 6 classes.
Found 3000 validated image filenames belonging to 6 classes.


In [13]:
classes=list(train.class_indices.keys())
print (classes)

['building', 'forest', 'glacier', 'mountain', 'sea', 'street']


In [24]:
from tensorflow.keras.applications import VGG16

cnn_base = VGG16(include_top = False,
                 weights = 'imagenet',
                 input_shape = (150, 150, 3),
                 classes = 6,
                 classifier_activation = 'softmax')

cnn_base.trainable = False

In [25]:
pretrainedCNN_model = Sequential([
                                     cnn_base,
                                  layers.Flatten(),
                                  layers.Dense(640, activation = 'relu'),
                                  layers.Dropout(0.5),
                                  layers.Dense(128, activation = 'relu'),
                                  layers.Dropout(0.5),
                                  layers.Dense(6, activation = 'softmax')  ,  
])

In [32]:

pretrainedCNN_model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
                            optimizer = keras.optimizers.Adam(0.0001),
                            metrics = ['acc'])

In [33]:
pretrainedCNN_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_2 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 640)               5243520   
_________________________________________________________________
dropout_4 (Dropout)          (None, 640)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 128)               82048     
_________________________________________________________________
dropout_5 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 6)                

In [34]:
History = pretrainedCNN_model.fit(train, validation_data= val, epochs=10,verbose=1,
                                  steps_per_epoch=1000//32,
                                  validation_steps=100//32)

Epoch 1/10
439/439 [==============================] - 95s 215ms/step - loss: 1.2804 - acc: 0.4900 - val_loss: 0.7676 - val_acc: 0.7223
Epoch 2/10
439/439 [==============================] - 90s 205ms/step - loss: 0.8514 - acc: 0.6652 - val_loss: 0.6239 - val_acc: 0.7753
Epoch 3/10
439/439 [==============================] - 90s 205ms/step - loss: 0.7574 - acc: 0.7064 - val_loss: 0.5890 - val_acc: 0.7780
Epoch 4/10
439/439 [==============================] - 90s 205ms/step - loss: 0.6628 - acc: 0.7520 - val_loss: 0.5867 - val_acc: 0.7737
Epoch 5/10
439/439 [==============================] - 89s 204ms/step - loss: 0.6538 - acc: 0.7583 - val_loss: 0.5256 - val_acc: 0.8047
Epoch 6/10
439/439 [==============================] - 89s 204ms/step - loss: 0.6263 - acc: 0.7671 - val_loss: 0.5162 - val_acc: 0.8097
Epoch 7/10
439/439 [==============================] - 89s 204ms/step - loss: 0.5952 - acc: 0.7806 - val_loss: 0.4870 - val_acc: 0.8203
Epoch 8/10
439/439 [==============================] - 8